In [ ]:
!pip install catboost jsonlines

  Using cached catboost-1.2.7-cp310-cp310-manylinux2014_x86_64.whl.metadata (1.2 kB)
Using cached catboost-1.2.7-cp310-cp310-manylinux2014_x86_64.whl (98.7 MB)


In [10]:
import json

# Файлы ввода
file1 = "/content/shingle_stats.json"
file2 = "/content/split_file_0.jsonl"
output_file = "output.json"

shingle_to_target = {}

with open(file2, "r", encoding="utf-8") as f2:
    for line in f2:
        data = json.loads(line)
        target = data["target"]
        for shingle in data["shingles"]:
            shingle_to_target[shingle] = target


result = []

with open(file1, "r", encoding="utf-8") as f1:
    for line in f1:
        record = json.loads(line.strip())
        shingle = record["shingle"]
        if shingle in shingle_to_target:
            record["target"] = shingle_to_target[shingle]
        result.append(record)

with open(output_file, "w", encoding="utf-8") as f_out:
    for item in result:
        f_out.write(json.dumps(item, ensure_ascii=False) + "\n")

print(f"ОК, все в {output_file}")


JSONDecodeError: Extra data: line 1 column 38 (char 37)

In [12]:
import jsonlines
import pandas as pd
from collections import defaultdict
from catboost import CatBoostClassifier, Pool
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

data_file = '/content/split_file_0.jsonl'
shingle_counts = defaultdict(int)
shingle_phishing_counts = defaultdict(int)

with jsonlines.open(data_file) as reader:
    for obj in reader:
        num = obj['num']
        target = obj['target']
        for shingle in obj['shingles']:
            shingle_counts[shingle] += num
            if target == 1:
                shingle_phishing_counts[shingle] += num

shingle_data = []
for shingle, count in shingle_counts.items():
    phishing_count = shingle_phishing_counts[shingle]
    phishing_ratio = phishing_count / count if count > 0 else 0
    group = int(phishing_ratio * 5)
    shingle_data.append((shingle, phishing_ratio, group))

In [13]:
shingle_df = pd.DataFrame(shingle_data, columns=["shingle", "phishing_ratio", "group"])
X = shingle_df[['phishing_ratio', 'group']]
y = shingle_df['shingle'].apply(lambda x: 1 if x in shingle_phishing_counts and shingle_phishing_counts[x] > 0 else 0)
X['group'] = X['group'].astype('category')

<ipython-input-13-cebea401c0ed>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X['group'] = X['group'].astype('category')


In [14]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

train_data = Pool(X_train, label=y_train, cat_features=[1])
test_data = Pool(X_test, label=y_test, cat_features=[1])

In [22]:
import matplotlib.pyplot as plt
import xgboost as xgb
import lightgbm as lgb
from catboost import CatBoostClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, roc_auc_score

def compute_metrics(y_true, y_pred, y_prob):
    return {
        'Accuracy': accuracy_score(y_true, y_pred),
        'Precision': precision_score(y_true, y_pred),
        'Recall': recall_score(y_true, y_pred),
        'ROC-AUC': roc_auc_score(y_true, y_prob),
    }

results = {}


catboost_model = CatBoostClassifier(iterations=50, learning_rate=0.1, depth=6, cat_features=[1], verbose=False)
catboost_model.fit(train_data, eval_set=test_data)
catboost_pred = (catboost_model.predict(X_test) > 0.5).astype(int)
catboost_prob = catboost_model.predict_proba(X_test)[:, 1]
results['CatBoost'] = compute_metrics(y_test, catboost_pred, catboost_prob)


xgboost_model = xgb.XGBClassifier(
    n_estimators=50,
    learning_rate=0.1,
    max_depth=6,
    use_label_encoder=False,
    eval_metric='logloss',
    enable_categorical=True
)
xgboost_model.fit(X_train, y_train, eval_set=[(X_test, y_test)], verbose=False)
xgboost_pred = (xgboost_model.predict(X_test) > 0.5).astype(int)
xgboost_prob = xgboost_model.predict_proba(X_test)[:, 1]
results['XgBoost'] = compute_metrics(y_test, catboost_pred, catboost_prob)

/usr/local/lib/python3.10/dist-packages/xgboost/core.py:158: UserWarning: [19:47:57] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


In [23]:
for model_name, metrics in results.items():
    print(f"Metrics for {model_name}:")
    for metric_name, value in metrics.items():
        print(f"  {metric_name}: {value:.4f}")
    print()

metrics_labels = ['Accuracy', 'Precision', 'Recall', 'ROC-AUC']
x = range(len(metrics_labels))

#plt.figure(figsize=(10, 6))

#for model_name, metrics in results.items():
#    plt.plot(x, [metrics[label] for label in metrics_labels], label=model_name, marker='o')

#plt.title('Model Comparison: Metrics After Training')
#plt.xticks(x, metrics_labels)
#plt.ylabel('Score')
#plt.ylim(0, 1)
#plt.legend()
#plt.grid()
#plt.show()

Metrics for CatBoost:
  Accuracy: 1.0000
  Precision: 1.0000
  Recall: 1.0000
  ROC-AUC: 1.0000

Metrics for XgBoost:
  Accuracy: 1.0000
  Precision: 1.0000
  Recall: 1.0000
  ROC-AUC: 1.0000

